In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Version history
* v.1 Started
* v.2 Added correlations of annual product sales to each of the GDP variants

# Introduction

In this competition, many people resort to external data for additional features. A popular choice is GDP data. However, there are several (free) venues to obtain such data. As quality of such data has an impact on the final forecast, I decided to perform a sanity check on the data from various sources. In the following, I compare 4 reputable sources:

* [UN data](https://data.un.org/)
* [IMF](https://data.imf.org/)
* [OECD](https://data.oecd.org/)
* [World Bank](https://data.worldbank.org/)


In [ ]:
from matplotlib import pyplot as plt

First, I have to combine data from these sources into one CSV file. Some sources have multiple ways to calculate the GDP, so a choice needs to be made. Generally, if a source doesn't specify, I'd assume, or if a source offers multiple ways, I'd choose, that the method of calculation is the "expenditure approach". Now, granted that, there are still 4 different GDP numbers. There are the nominal GDP and the real GDP (inflation adjusted), and each can be expressed in local currency (LCU) or USD. If it is in USD, then I'd either assume (if the source does not specify) or choose the method such that the exchange rate for the nominal GDP is the current rate for the year, while the exchange rate for the real GDP is a constant rate for the base year that has been used for the inflation adjustment.

In [ ]:
df = pd.read_csv('../input/gdp-fin-nor-swe-20152019-multiple-sources/GDP_FIN_NOR_SWE_2015-2019_Multiple_Sources.csv')

In [ ]:
df

In [ ]:
def get_gdp(df, country,nominal,local_currency,data_source):
    if nominal:
        if local_currency:
            measure = 'Current prices'
        else:
            measure = 'Current prices, current exchange rates'
    else:
        if local_currency:
            measure = 'Constant prices'
        else:
            measure = 'Constant prices, constant exchange rates'
    df = df[(df['Country']==country)&(df['Measure']==measure)&(df['Data Source']==data_source)].copy()
    if local_currency:
        df = df[df['Currency (millions)']!='USD']
    else:
        df = df[df['Currency (millions)']=='USD']
    
    if len(df) == 0:
        raise 
    else:
        df = df.sort_values(by='Year').reset_index().drop(['index'],axis=1)
        return df

As an example, the following code is how we can get the Finland nominal GDP in local currency (Euro) from UN data. There is no base year in this case.

In [ ]:
get_gdp(df,'Finland',True,True,'UN data')

Now let's plot some graphs for comparing the nominal GDP data from different sources.

# Nominal GDP in local currency

In [ ]:
from matplotlib.ticker import MaxNLocator

fig = plt.figure(figsize=(24, 8))
for i,country in enumerate(['Finland','Norway','Sweden']):
    fig.add_subplot(1,3,i+1)
    plt.plot(get_gdp(df,country,True,True,'UN data').Year,get_gdp(df,country,True,True,'UN data').Value, 'bx-')
    plt.plot(get_gdp(df,country,True,True,'IMF').Year,get_gdp(df,country,True,True,'IMF').Value, 'mo-')
    plt.plot(get_gdp(df,country,True,True,'OECD').Year,get_gdp(df,country,True,True,'OECD').Value, 'g*-')
    plt.plot(get_gdp(df,country,True,True,'World Bank').Year,get_gdp(df,country,True,True,'World Bank').Value, 'r+-')
    plt.legend(['UN data', 'IMF','OECD','World Bank'], loc='upper left')
    plt.title(F"{country} GDP (nominal, {get_gdp(df,country,True,True,'UN data')['Currency (millions)'].iloc[0]})")
    plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

plt.show()

It looks like the data mostly agree with each other. There are some discrepancies at year 2019, which is unfortunate because we need to use this data point to aid us in our forecast.

# Nominal GDP in USD

Only OECD and World Bank provide GDP data in USD.

In [ ]:
fig = plt.figure(figsize=(24, 8))
for i,country in enumerate(['Finland','Norway','Sweden']):
    fig.add_subplot(1,3,i+1)
    plt.plot(get_gdp(df,country,True,False,'OECD').Year,get_gdp(df,country,True,False,'OECD').Value, 'g*-')
    plt.plot(get_gdp(df,country,True,False,'World Bank').Year,get_gdp(df,country,True,False,'World Bank').Value, 'r+-')
    plt.legend(['OECD','World Bank'], loc='upper left')
    plt.title(F"{country} GDP (nominal, {get_gdp(df,country,True,False,'OECD')['Currency (millions)'].iloc[0]})")
    plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

plt.show()

Again, some minor discrepancies at year 2019. But these graphs differ from the ones for local currency in a big way. Whereas the GDP graphs are trending up for every year (although there is an obvious slow down for Norway in 2019), GDP for all countries dipped drastically in 2019 when in USD. We'll take a closer look in the later section, but it would appear that this dip is an exchange rate artifact.

# Real GDP in local currency

In [ ]:
from matplotlib.ticker import MaxNLocator

fig = plt.figure(figsize=(24, 8))
for i,country in enumerate(['Finland','Norway','Sweden']):
    fig.add_subplot(1,3,i+1)
    plt.plot(get_gdp(df,country,False,True,'UN data').Year,get_gdp(df,country,False,True,'UN data').Value, 'bx-')
    plt.plot(get_gdp(df,country,False,True,'IMF').Year,get_gdp(df,country,False,True,'IMF').Value, 'mo-')
    plt.plot(get_gdp(df,country,False,True,'OECD').Year,get_gdp(df,country,False,True,'OECD').Value, 'g*-')
    plt.plot(get_gdp(df,country,False,True,'World Bank').Year,get_gdp(df,country,False,True,'World Bank').Value, 'r+-')
    legend = []
    for src in ['UN data', 'IMF','OECD','World Bank']:
        legend.append(src + F' (base year {get_gdp(df,country,False,True,src)["Base Year"].iloc[0]})')
    plt.legend(legend, loc='upper left')
    plt.title(F"{country} GDP (real, {get_gdp(df,country,False,True,'UN data')['Currency (millions)'].iloc[0]})")
    plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

plt.show()

Now here it is clear that the data from IMF is "different". But it is all explainable. For real GDP, data are inflation adjusted using a base year. All data except the IMF data use a base year of 2015, while IMF data uses a base year of 2010. As the 2010 prices are lower, it makes sense that the real GDP based on 2010 prices should be lower.

# Real GDP in USD

In [ ]:
fig = plt.figure(figsize=(24, 8))
for i,country in enumerate(['Finland','Norway','Sweden']):
    fig.add_subplot(1,3,i+1)
    plt.plot(get_gdp(df,country,False,False,'OECD').Year,get_gdp(df,country,False,False,'OECD').Value, 'g*-')
    plt.plot(get_gdp(df,country,False,False,'World Bank').Year,get_gdp(df,country,False,False,'World Bank').Value, 'r+-')
    legend = []
    for src in ['OECD','World Bank']:
        legend.append(src + F' (base year {get_gdp(df,country,False,True,src)["Base Year"].iloc[0]})')
    plt.legend(legend, loc='upper left')
    plt.title(F"{country} GDP (real, {get_gdp(df,country,False,False,'OECD')['Currency (millions)'].iloc[0]})")
    plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

plt.show()

Here we see the expected discrepany in 2019 again, but there is no dip in 2019. We will see why in a minute. It has to do with using a fixed exchange rate based at year 2015.

# Exchange rates

Let's stick with one data source, say World Bank. We can figure out the the exchange rates they used to calculate the GDP in USD.

### Nominal GDP

In [ ]:
df1 = df[df['Data Source']=='World Bank'].copy().set_index(['Year','Country'])
df1=df1[df1['Measure']=='Current prices'][['Value']].join(df1[df1['Measure']=='Current prices, current exchange rates'][['Value']],lsuffix='_lcu',rsuffix='_usd')
df1['Exchange Rate (USD)']=df1['Value_usd']/df1['Value_lcu']
df1

It seems that the relative strength of USD in 2019 is causing the GDP's of these countries, which had positive, albeit slowing growth, to appear as declines when expressed in USD.

### Real GDP

In [ ]:
df2 = df[df['Data Source']=='World Bank'].copy().set_index(['Year','Country'])
df2=df2[df2['Measure']=='Constant prices'][['Value']].join(df2[df2['Measure']=='Constant prices, constant exchange rates'][['Value']],lsuffix='_lcu',rsuffix='_usd')
df2['Exchange Rate (USD)']=df2['Value_usd']/df2['Value_lcu']
df2

This "exchange rate artifact" does not exist in the real GDP because a fixed exchange rate (based at 2015) is used.

# Which GDP should we use?

This is the burning question for this competition, isn't it? The following is just my opinion. 

From a logical point of view, unless the shops price their products in USD, the GDP in local currency should be used. If they priced in USD and the USD became stronger, then indeed the products would appear as more expensive to the locals, and the number of items sold would decrease, following what the GDP in USD would suggest. Also, as we are forecasting the number of items sold, not the sales in currency units, the real GDP, which is inflation adjusted, should be used instead of the nominal GDP. This is because the item price should go up with inflation, so the number of items sold would stay the same if the real GDP stayed flat, for example.

From the point of view of this competition, whichever GDP gives a higher score on the LB should be the one to use :-).  Other than (over-)fitting to the LB, we can also try to determine which one to use by correlating annual product sales (in number of units sold) to the each of the GDP's from 2015 to 2018. 

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv',parse_dates=['date'])
train_data['year'] = train_data.date.apply(lambda x:x.year)

In [ ]:
corr_table = []
for country in ['Finland','Norway','Sweden']:
    for store in ['KaggleMart','KaggleRama']:
        for product in ['Kaggle Hat','Kaggle Mug', 'Kaggle Sticker']:
            row = [country,store,product]
            yearly_num_sold_series = train_data.groupby(['country','store','product','year'])['num_sold'].sum()[(country,store,product)]
            for nominal in [True,False]:
                for local_currency in [False,True]:
                    gdp_series = get_gdp(df,country,nominal,local_currency,'World Bank').set_index(['Year'])['Value'].iloc[:-1]
                    row.append(yearly_num_sold_series.corr(gdp_series))
            corr_table.append(row)
            
corr_df = pd.DataFrame(corr_table,columns=['country','store','product','nominal GDP (USD)','nominal GDP (LCU)','real GDP (USD)', 'real GDP (LCU)'])
corr_df['best correlated'] = np.array(['nominal GDP (USD)','nominal GDP (LCU)','real GDP (USD)', 'real GDP (LCU)'])[np.argmax(corr_df.iloc[:,-4:].to_numpy(),axis=1)]
corr_df

Based on this correlation table, except in couple of isolated cases (KaggleRama in Norway), nominal GDP in USD correlates best with annual product sales. Well, so much for common sense economic reasoning.